In [99]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [100]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [101]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [102]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [103]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(df_market_data)

In [104]:
# Create a DataFrame with the scaled data
market_scaled_df = pd.DataFrame(market_scaled, index=df_market_data.index, columns=df_market_data.columns)

# Display sample data
market_scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [105]:
# Create a list with the number of k-values from 1 to 11
k_values= list(range(1,11))

In [106]:
# Create an empty list to store the inertia values
inertia_list = []

# Create a for loop to compute the inertia with each possible value of k

for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state= 42)
    # 2. Fit the model to the data using `df_market_data_scaled`
    model.fit(market_scaled_df)    
    # 3. Append the model.inertia_ to the inertia list
    inertia_list.append(model.inertia_)

# Optional: Print the inertia values to inspect
for k, inertia in zip(k_values, inertia_list):
    print(f"k={k}: Inertia={inertia}")

k=1: Inertia=287.0
k=2: Inertia=212.1233420748626
k=3: Inertia=170.19357087970297
k=4: Inertia=79.02243535120975
k=5: Inertia=71.0979135124442
k=6: Inertia=66.10882678044752
k=7: Inertia=54.48812315624117
k=8: Inertia=43.22469266658042
k=9: Inertia=32.86088447122681
k=10: Inertia=28.404884636873092


In [107]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data={"k":k_values,"inertia":inertia_list}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow= pd.DataFrame(elbow_data)

In [108]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_1 = df_elbow.hvplot.line(
                x="k", 
                y="inertia", 
                title="Elbow Curve", 
                xticks=k
                )
elbow_plot_1

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [109]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4,random_state= 1,n_init=10)

In [110]:
# Fit the K-Means model using the scaled data
model.fit(market_scaled_df)

KMeans(n_clusters=4, n_init=10, random_state=1)

In [111]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(market_scaled_df)

# Print the resulting array of cluster values.
print(k_4)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 3 2 0 0 1
 0 0 0 0]


In [112]:
# Create a copy of the DataFrame
df_market_predictions= market_scaled_df.copy()

In [113]:
# Add a new column to the DataFrame with the predicted clusters
df_market_predictions["predicted_clusters"]=k_4

# Display sample data
df_market_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [114]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_predictions_plot = df_market_predictions.hvplot.scatter(
                                    x="price_change_percentage_24h",
                                    y="price_change_percentage_7d",
                                    by="predicted_clusters",
                                    hover_cols = ["coin_id"],
                                    title = "Market Data Predictions"
                                )
market_data_predictions_plot

:NdOverlay   [predicted_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [115]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [116]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(market_scaled_df)
# View the first five rows of the DataFrame. 
market_data_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [117]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

In [118]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
market_data_pca_df = pd.DataFrame(market_data_pca,columns = ["PCA1","PCA2","PCA3"])
# Copy the crypto names from the original data
market_data_pca_df["coin_id"] = df_market_data.index

# Set the coinid column as index
market_data_pca_df= market_data_pca_df.set_index("coin_id")

# Display sample data
market_data_pca_df.head()

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [119]:
# Create a list with the number of k-values from 1 to 11
k_values= list(range(1,11))

In [120]:
# Create an empty list to store the inertia values
inertia_list = []
# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=42,n_init =10)
# 2. Fit the model to the data using `df_market_data_pca`
    model.fit(market_data_pca_df)
# 3. Append the model.inertia_ to the inertia list
    inertia_list.append(model.inertia_)


In [121]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data= {"k":k_values,"inertia":inertia_list}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow =  pd.DataFrame(elbow_data)

In [122]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_2 = df_elbow.hvplot.line(
                x="k", 
                y="inertia", 
                title="Elbow Curve PCA", 
                xticks=k
                )
elbow_plot_2

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** no

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [123]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters= 4,random_state=42, n_init=10)

In [124]:
# Fit the K-Means model using the PCA data
model.fit(market_data_pca_df)

KMeans(n_clusters=4, n_init=10, random_state=42)

In [125]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_data_pca_predictions = model.predict(market_data_pca_df)
# Print the resulting array of cluster values.
print(market_data_pca_predictions)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [126]:
# Create a copy of the DataFrame with the PCA data
market_data_pca_predictions_df = market_data_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters

market_data_pca_predictions_df["predicted_clusters"]= market_data_pca_predictions
# Display sample data
market_data_pca_predictions_df.head()

,PCA1,PCA2,PCA3,predicted_clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,2
ripple,-0.471835,-0.222660,-0.479053,2
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [127]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_data_pca_predictions_plot = market_data_pca_predictions_df.hvplot.scatter(
                                        x="PCA1",
                                        y="PCA2",
                                        by="predicted_clusters",
                                        hover_cols = ["coin_id"],
                                        title= "Market Data PCA Predictions"

                                    )
market_data_pca_predictions_plot

:NdOverlay   [predicted_clusters]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [128]:
# Composite plot to contrast the Elbow curves
(elbow_plot_1+elbow_plot_2).cols(1)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [129]:
# Composite plot to contrast the clusters
(market_data_predictions_plot + market_data_pca_predictions_plot).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [predicted_clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_clusters]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features to cluster data often results in simpler, more interpretable clusters but may overlook complex relationships between features, potentially reducing the granularity or accuracy of the clustering.